In [11]:
import time
import sys
import pandas as pd
import numpy as np
from src.process_bank_churn import preprocess_data, preprocess_new_data
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [12]:
raw_df = pd.read_csv("bank-customer-churn-prediction-dlu-course-c-2/train.csv")

In [13]:
proc_data = preprocess_data(raw_df, 'Exited', ["id", "CustomerId"])
proc_data.keys()

dict_keys(['train_X', 'train_y', 'test_X', 'test_y', 'input_cols', 'target_col', 'scaler', 'encoder'])

1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [14]:
knn = KNeighborsClassifier()
knn.fit(proc_data['train_X'], proc_data['train_y'])

roc_auc_train = roc_auc_score(proc_data['train_y'], knn.predict_proba(proc_data['train_X'])[:, 1])
roc_auc_test = roc_auc_score(proc_data['test_y'], knn.predict_proba(proc_data['test_X'])[:, 1])

roc_auc_train, roc_auc_test

(np.float64(0.8237648065802492), np.float64(0.5734031826599904))

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [15]:
param_grid = {
    'n_neighbors': range(1, 21)
}

grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='roc_auc')
grid_search.fit(proc_data['train_X'], proc_data['train_y'])

print("Найкращий гіперпараметр:", grid_search.best_params_)
knn_best = grid_search.best_estimator_

Найкращий гіперпараметр: {'n_neighbors': 20}


In [16]:
roc_auc_train = roc_auc_score(proc_data['train_y'], knn_best.predict_proba(proc_data['train_X'])[:, 1])
roc_auc_test = roc_auc_score(proc_data['test_y'], knn_best.predict_proba(proc_data['test_X'])[:, 1])

roc_auc_train, roc_auc_test

(np.float64(0.696798172080658), np.float64(0.6106886617737841))

**Conclusion**

It seems like the model was overfitted before, as the generalization was very poor. However, the Grid Search helped to find optimal parameters that made the model more stable and better.

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [17]:
dt = DecisionTreeClassifier(random_state=42)

grid_params = {
    'max_depth': range(1, 21, 2),
    'max_leaf_nodes': range(2, 11)
}

grid_search = GridSearchCV(dt, grid_params, cv=3, scoring='roc_auc')

start_time = time.time()
grid_search.fit(proc_data['train_X'], proc_data['train_y'])
end_time = time.time()
exec_time = end_time - start_time

dt_best = grid_search.best_estimator_
print("Найкращий гіперпараметр:", grid_search.best_params_)

roc_auc_train = roc_auc_score(proc_data['train_y'], dt_best.predict_proba(proc_data['train_X'])[:, 1])
roc_auc_test = roc_auc_score(proc_data['test_y'], dt_best.predict_proba(proc_data['test_X'])[:, 1])

print(f"Час виконання: {exec_time:.2f} секунд")
print(f"AUROC на тренувальній вибірці: {roc_auc_train:.4f}")
print(f"AUROC на валідаційній вибірці: {roc_auc_test:.4f}")

Найкращий гіперпараметр: {'max_depth': 5, 'max_leaf_nodes': 10}
Час виконання: 2.27 секунд
AUROC на тренувальній вибірці: 0.9015
AUROC на валідаційній вибірці: 0.9002


**Висновок**
- Модель, знайдена за допомогою GridSearchCV, виявилася кращою.

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [18]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

dt = DecisionTreeClassifier(random_state=42)

random_search = RandomizedSearchCV(dt, params_dt, n_iter=40, cv=3, scoring='roc_auc', random_state=42)

start_time = time.time()
random_search.fit(proc_data['train_X'], proc_data['train_y'])
end_time = time.time()
exec_time = end_time - start_time

dt_random_search_best = random_search.best_estimator_
print("Найкращий гіперпараметр:", random_search.best_params_)

y_train_pred = dt_random_search_best.predict_proba(proc_data['train_X'])[:, 1]
y_test_pred = dt_random_search_best.predict_proba(proc_data['test_X'])[:, 1]

roc_auc_train = roc_auc_score(proc_data['train_y'], y_train_pred)
roc_auc_test = roc_auc_score(proc_data['test_y'], y_test_pred)

print(f"Час виконання: {exec_time:.2f} секунд")
print(f"AUROC на тренувальній вибірці: {roc_auc_train:.4f}")
print(f"AUROC на валідаційній вибірці: {roc_auc_test:.4f}")

Найкращий гіперпараметр: {'splitter': 'best', 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_leaf_nodes': np.int64(14), 'max_features': None, 'max_depth': np.int64(16), 'criterion': 'entropy'}
Час виконання: 0.54 секунд
AUROC на тренувальній вибірці: 0.9169
AUROC на валідаційній вибірці: 0.9166


**Висновок**
- Модель, знайдена за допомогою RandomizedSearchCV, має вищу якість, оскільки AUROC на тренувальній (0.9169) і валідаційній (0.9166) вибірках перевищує відповідні значення для GridSearchCV (0.9015 та 0.9002).
- RandomizedSearchCV виглядає більш ефективною, оскільки вона краще узагальнює дані (вищий AUROC) і при цьому використовує більш гнучку архітектуру.

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [19]:
test_raw_df = pd.read_csv("bank-customer-churn-prediction-dlu-course-c-2/test.csv")

test_X = preprocess_new_data(test_raw_df, proc_data['encoder'], proc_data['scaler'], ["id", "CustomerId"])
prediction_probs = dt_random_search_best.predict_proba(test_X)[:, 1]

# Формування submission.csv
sample_raw_df = pd.DataFrame({'id': test_raw_df["id"].values})
sample_raw_df['Exited'] = prediction_probs
sample_raw_df.to_csv("bank-customer-churn-prediction-dlu-course-c-2/submission_log_reg.csv", index=False)